In [1]:
###################################################################################################
# WPS_ERA5_PREPROCESSOR.ipynb
# ATM 419/563 Fall 2024
# 5 December 2024 Robert Fovell (rfovell@albany.edu) 
#
#
# Notebook to process ERA5 reanalysis fields in NetCDF format from NCAR RDA into intermediate format
# based on Luke Madaus' wrf-preproc-intermediate-writer, with substantial modifications for ERA5
# [https://gitlab.com/jupiter-opensource/wrf-preproc-intermediate-writer]
#
# No warranty, no promises, no support guarantee
###################################################################################################

'''

This is tested on ERA5 NetCDF files from NCAR RDA.  Will need (substantial) modifications to work with other sources.
Geographic subsetting does not currently work.  The routine process_soil_levels is not being used in this version.

Files needed besides this notebook:
• ERA5_RH2m_computer.ipynb       - to create 2 m RH
• era5_SOILHGT.nc (ERA5 terrain) - needed for proper WRF initialization

The processor software location is defined below, defaulting to /network/rit/lab/atm419lab/SOFTWARE/ERA5_PREPROCESSOR/
The contents of that directory are
• wps_formatter_ERA5_RGF.py [modified from Madaus' code]
• grib_codes.df.pkl         [from Luke Madaus]
• __init__.py (which links to wps_formatter_ERA5_RGF.py

----------------------------------------------------------------------------------------------
BEFORE RUNNING THIS NOTEBOOK:
----------------------------------------------------------------------------------------------

(1) Download ERA5 files from NCAR RDA in NetCDF format.
  Isobaric level fields needed: z, t, u, v, r.  (q is not needed)
  Surface fields needed: msl, sp, 2t, sstk, skt, 10u, 10v, 2d
  Subsurface fields needed: stl1, stl2, stl3, stl4, swvl1, swvl2, swvl3, swvl4
  
(2) For multiple day runs, combine isobaric level files by field.  This can take a lot of time.

ncrcat e5.oper.an.pl.128_129_z* era5_z.nc
ncrcat e5.oper.an.pl.128_130_t* era5_t.nc
ncrcat e5.oper.an.pl.128_131_u* era5_u.nc
ncrcat e5.oper.an.pl.128_132_v* era5_v.nc
ncrcat e5.oper.an.pl.128_157_r* era5_r.nc

(3) The surface fields each encompass an entire month.  For runs crossing months, you may need to 
 combine surface files or run this notebook and do the following steps more than once

(4) ERA5 does not supply 2 m RH.  Not having 2 m RH can also cause real.exe to ignore U10 and V10 information,
 changing fields near the surface, because it forces use_surface to False 
    + Run notebook ERA5_RH2m_computer.ipynb to read in 2t, 2d files, computing 2m RH via MetPy
    + This generates a new NetCDF file called output_file.nc
    + Then change name of variable in output file:  ncrename -v VAR_2D,VAR_2RH output_file.nc
    + Change name of output_file.nc if needed:      mv output_file.nc era5_rh2m_aug2019.nc
    
(5) Have the invariant field SOILHGT available and expressed in meters (not surface geopotential in m2/s2).
 The file /network/rit/lab/atm419lab/DATA/ERA5/era5_SOILHGT.nc is ready for use
 
(6) Modify Cell #2 for locations, names of files

(7) Also in Cell #2, point to location of the grib_codes.df.pkl file

(8) In Cell #3, set the time range.  Example:
'time_range' : (datetime(2019,8,26,12), datetime(2019,8,27,0)),

(9) In Cell #3, set the time interval (interval_seconds, expressed in hours).  Example:
'time_freq' : '3h',

(10) In the metgrid step, do NOT include the ERA5 invariant file that you would use with GRIB data.
This appears to corrupt the soils information.

        
'''
%env PROJ_DATA=/network/rit/lab/snowclus/miniforge3/envs/jan24_env/share/proj

# -------------------------------------------------------------------------------------------------------
# WPS Formatter
# declare where the modified wps_formatter code can be found:
import sys
sys.path.append("/network/rit/lab/atm419lab/SOFTWARE/ERA5_PREPROCESSOR/")  # Adds the directory to Python's search path
import wps_formatter_ERA5_RGF
from wps_formatter_ERA5_RGF import WRFInputFormatter
# -------------------------------------------------------------------------------------------------------

from datetime import datetime,timedelta
import pandas as pd
import xarray as xr
from netCDF4 import Dataset
import pickle

print(" ready to go...")

env: PROJ_DATA=/network/rit/lab/snowclus/miniforge3/envs/jan24_env/share/proj
 ready to go...


In [2]:
# These paths point to which netcdf file that contain each variable
variable_paths = {
    
    # Invariant
    'SOILHGT' : '/network/rit/lab/atm419lab/DATA/ERA5/era5_SOILHGT.nc',
    
    # Surface variables. VAR_2RH file created with ERA5_RH2m_computer.ipynb
    'MSL' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_151_msl.ll025sc.2019080100_2019083123.nc',
    'SP' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_134_sp.ll025sc.2019080100_2019083123.nc',
    'VAR_2T' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_167_2t.ll025sc.2019080100_2019083123.nc',
    'SSTK' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_034_sstk.ll025sc.2019080100_2019083123.nc',
    'SKT' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_235_skt.ll025sc.2019080100_2019083123.nc',
    'VAR_10U' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_165_10u.ll025sc.2019080100_2019083123.nc',
    'VAR_10V' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_166_10v.ll025sc.2019080100_2019083123.nc',
    'VAR_2RH' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/era5_rh2m_aug2019.nc',
    
    # Upper air variables. Z is geopotential, not geopotential height
    'T' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/era5_t.nc',
    'R' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/era5_r.nc',
    'U' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/era5_u.nc',
    'V' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/era5_v.nc',
    'Z' : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/era5_z.nc',
   
    # Soil variables
    'STL1'   : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_139_stl1.ll025sc.2019080100_2019083123.nc',
    'STL2'   : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_170_stl2.ll025sc.2019080100_2019083123.nc',
    'STL3'   : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_183_stl3.ll025sc.2019080100_2019083123.nc',
    'STL4'   : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_236_stl4.ll025sc.2019080100_2019083123.nc',
    'SWVL1'   : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_039_swvl1.ll025sc.2019080100_2019083123.nc',
    'SWVL2'   : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_040_swvl2.ll025sc.2019080100_2019083123.nc',
    'SWVL3'   : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_041_swvl3.ll025sc.2019080100_2019083123.nc',
    'SWVL4'   : '/network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_042_swvl4.ll025sc.2019080100_2019083123.nc',
}
# This table links variable names in the original file with their equivalent
# names in WPS format
grib_codes = pd.read_pickle('/network/rit/lab/atm419lab/SOFTWARE/ERA5_PREPROCESSOR/grib_codes.df.pkl')
#grib_codes.head()

# -----------------------------------------------------------------------------------------------
# Editing below this line may not be necessary
# -----------------------------------------------------------------------------------------------

# add new rows for ERA5 variable names
# the grib codes do not appear to matter at all.  The metgrid names are crucial.
# we could pickle this, of course
#                       ['Field description ',gribcode,levelcode,'metgrid name','units']
grib_codes.loc['MSL'] = ['Sea-level Pressure', 2, 102,'PMSL','Pa']
grib_codes.loc['SP'] = ['Surface Pressure', 1, 1,'PSFC','Pa']
grib_codes.loc['VAR_2T'] = ['Temperature at 2 m', 11, 105,'TT','K']
grib_codes.loc['VAR_2D'] = ['Dewpoint Temperature at 2 m', 17, 105,'DPT','K']
grib_codes.loc['VAR_2RH'] = ['Relative humidity at 2 m', 52, 105,'RH','%']
grib_codes.loc['VAR_10U'] = ['U at 10 m', 33, 105,'UU','m s-1']
grib_codes.loc['VAR_10V'] = ['V at 10 m', 34, 105,'VV','m s-1']
grib_codes.loc['STL1'] = ['T of 0-7 cm ground layer', 139, 112,'ST000007','K']
grib_codes.loc['STL2'] = ['T of 7-28 cm ground layer', 170, 112,'ST007028','K']
grib_codes.loc['STL3'] = ['T of 28-100 cm ground layer', 183, 112,'ST028100','K']
grib_codes.loc['STL4'] = ['T of 100-289 cm ground layer', 236, 112,'ST100289','K']
grib_codes.loc['SWVL1'] = ['Soil moisture of 0-7 cm ground layer', 39, 112,'SM000007','fraction']
grib_codes.loc['SWVL2'] = ['Soil moisture of 7-28 cm ground layer', 40, 112,'SM007028','fraction']
grib_codes.loc['SWVL3'] = ['Soil moisture of 28-100 cm ground layer', 41, 112,'SM028100','fraction']
grib_codes.loc['SWVL4'] = ['Soil moisture of 100-289 cm ground layer', 42, 112,'SM100289','fraction']
grib_codes.loc['SKT'] = ['Skin Temperature', 11, 1,'SKINTEMP','K'] #Vtable.ECMWF gribcode is 235
grib_codes.loc['SSTK'] = ['Sea-Surface Temperature', 34, 1,'SST','K'] 
# Z is geopotential, and I am converting it to geopotential height
grib_codes.loc['Z'] = ['Geopotential height', 7, 100,'GHT','m'] 
grib_codes.loc['R'] = ['Relative Humidity',52,100,'RH','%'] #Vtable.ECMWF gribcode is 157
grib_codes.loc['SOILHGT'] = ['Terrain field of source analysis', 7, 1,'SOILHGT','m'] 

#print(" ------- ")
#print(grib_codes)
print(" ready...")

 ready...


In [3]:

# Start by supplying the variable_paths and the grib_codes table
params = {
    'variable_paths' : variable_paths,
    'variable_table' : grib_codes,
    
    # Subset to this time range (start, end)
    'time_range' : (datetime(2019,8,26,12), datetime(2019,8,27,0)),
    
    # We want output every this many hours.  This particular
    # string format is what xarray looks for
    'time_freq' : '3h',
    
    # Set the vertical coordinate type of the raw dataset.
    # Options are 'pressure', with limited support for 'hybrid-p' and 'hybrid-z'
    'vcoord_type' : 'pressure',
    
    # These are the names of the vertical coordinates in the datasets
    # For upper air pressure levels:
    'vcoord' : 'level',
    
    # And for subsurface soil levels
    'soilcoord' : 'depth',
    
    # Optional...set a geographical subset bounds in lat/lon
    # Lon in 0-365 space
    # If not provided, will return the entire extend of the netCDF file dataset
    #'geog_bounds' : {'south': 22, 'north': 52, 'west': 360-135, 'east': 360-55}, # DOES NOT WORK

}
print(" ready...")

 ready...


In [4]:
###############################################################################################
# The magic happens here
###############################################################################################

wrfgen = WRFInputFormatter(**params)

wrfgen.open_WPS_files()

for var in list(variable_paths.keys()):
    #print(" RGF variable ",var)
    varray = wrfgen.load_and_subset(var)
    wrfgen.add_to_WPS(varray)
    
wrfgen.close_WPS_files()
print(" done.")

2024-12-05 20:31:01,887 - WRFInputFormatter - INFO - THESE ARE GEOG BOUNDS: None
2024-12-05 20:31:01,887 - WRFInputFormatter - INFO - Building domain structure from pressure field
2024-12-05 20:31:01,888 - WRFInputFormatter - INFO - Loading and subsetting T
2024-12-05 20:31:01,888 - WRFInputFormatter - INFO - Loading variable T from: /network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/era5_t.nc
2024-12-05 20:31:04,698 - WRFInputFormatter - INFO - Resampling in time to requested frequency
2024-12-05 20:31:25,661 - WRFInputFormatter - WARNING - Suspect we have hPa...converting to Pa
2024-12-05 20:31:26,815 - WRFInputFormatter - INFO - Done with pressure
2024-12-05 20:31:26,816 - WRFInputFormatter - INFO - GEOG BOUNDS REQD: None
2024-12-05 20:31:26,816 - WRFInputFormatter - INFO - ACTUAL LATS: -90.0 -- 90.0
2024-12-05 20:31:26,816 - WRFInputFormatter - INFO - ACTUAL LONS: 0.0 -- 359.75
2024-12-05 20:31:26,817 - WRFInputFormatter - INFO - Done with initialization
2024-12-05 20:31:26,998 - 

 *** RGF single level variable SOILHGT
 *** RGF single level variable SOILHGT


2024-12-05 20:31:27,813 - WRFInputFormatter - INFO -    2019-08-26 18:00:00
2024-12-05 20:31:27,966 - WRFInputFormatter - INFO -    2019-08-26 21:00:00


 *** RGF single level variable SOILHGT
 *** RGF single level variable SOILHGT


2024-12-05 20:31:28,115 - WRFInputFormatter - INFO -    2019-08-27 00:00:00
2024-12-05 20:31:28,264 - WRFInputFormatter - INFO - Loading and subsetting MSL
2024-12-05 20:31:28,264 - WRFInputFormatter - INFO - Loading variable MSL from: /network/rit/lab/atm419lab/DATA/ERA5_AUG2019_NETCDF/e5.oper.an.sfc.128_151_msl.ll025sc.2019080100_2019083123.nc


...

 *** RGF single level variable SM100289
 *** RGF single level variable SM100289


2024-12-05 20:36:31,554 - WRFInputFormatter - INFO -    2019-08-26 18:00:00
2024-12-05 20:36:31,701 - WRFInputFormatter - INFO -    2019-08-26 21:00:00


 *** RGF single level variable SM100289
 *** RGF single level variable SM100289


2024-12-05 20:36:31,854 - WRFInputFormatter - INFO -    2019-08-27 00:00:00


 *** RGF single level variable SM100289
 done.
